<a href="https://colab.research.google.com/github/jseeun00/ESAA_2021-2/blob/main/5%EC%A3%BC%EC%B0%A8_%ED%95%84%EC%82%AC1_09_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# 원본 행렬 R(4 x 5) 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3으로 설정.
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

num_users, num_items = R.shape
K=3

# P와 Q 행렬의 크기를 지정하고, 정규 분포를 가진 임의의 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [12]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    # RMSE 계산
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse


In [13]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000  # SGD 업데이트 횟수
learning_rate=0.01  # 학습률
r_lambda=0.01  # L2 regularization 계수

# SGD 기법으로 P와 Q 행렬을 계속 업데이트.
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.106462973977251
### iteration step :  50  rmse :  0.411539926709536
### iteration step :  100  rmse :  0.14369586500864973
### iteration step :  150  rmse :  0.07004813501947217
### iteration step :  200  rmse :  0.041324020551916266
### iteration step :  250  rmse :  0.02834273266085604
### iteration step :  300  rmse :  0.02219191972286278
### iteration step :  350  rmse :  0.01929211204485788
### iteration step :  400  rmse :  0.01792726971865785
### iteration step :  450  rmse :  0.017273303454990275
### iteration step :  500  rmse :  0.01695033918959422
### iteration step :  550  rmse :  0.016784540431676143
### iteration step :  600  rmse :  0.0166943808655161
### iteration step :  650  rmse :  0.016640290014224523
### iteration step :  700  rmse :  0.016602536864005184
### iteration step :  750  rmse :  0.01657118732169371
### iteration step :  800  rmse :  0.016541315626967923
### iteration step :  850  rmse :  0.016510609679630747
### iterati

In [14]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.898 1.306 2.002 1.663]
 [6.694 4.978 0.978 2.981 1.003]
 [6.675 0.394 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.141]]
